In [37]:
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
from tensorflow.keras.preprocessing import image
import tensorflow.keras

In [38]:


import numpy as np 
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.resnet50 import ResNet50,preprocess_input
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from keras.utils import to_categorical
import os


In [39]:
print(os.listdir("../input/lunar-images/DataSet"))

['train.csv', 'Train Images', 'test.csv', 'Test Images']


In [40]:
training_csv=pd.read_csv("../input/lunar-images/DataSet/train.csv")
img_train_name=training_csv["Image_File"].values
y_train=training_csv["Class"].values

In [41]:
img_train_name

array(['clean2416.png', 'ground4022.png', 'ground0340.png', ...,
       'clean5172.png', 'ground1715.png', 'ground1250.png'], dtype=object)

In [42]:
count=0
for i in img_train_name:
    if i[:-8]=="clean":
        img_train_name[count]="../input/lunar-images/DataSet/Train Images/Train Images/Large/{}".format(i)
    else:
        img_train_name[count]="../input/lunar-images/DataSet/Train Images/Train Images/Small/{}".format(i)
    count=count+1

In [ ]:
# resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:

# resnet=ResNet50(include_top = False, pooling = "avg", weights = resnet_weights_path,input_shape=(227,227,3))

In [ ]:
# fc1 = Dense(512, activation='relu')(resnet.output)
# drp1=Dropout(0.20)(fc1)
# fc2 = Dense(256, activation='relu')(drp1)
# bat_norm=BatchNormalization()(fc2)
# fc3 = Dense(64, activation='relu')(bat_norm)
# out = Dense(2, activation='softmax')(fc3)

# model = Model(inputs=resnet.input,outputs=out)
# model.summary()

In [43]:
my_weight_init=tensorflow.keras.initializers.glorot_normal(seed=2)
model=Sequential()
model.add(Convolution2D(64,(3,3),input_shape=(227,227,3),kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2))

model.add(Convolution2D(128,(3,3),kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(Convolution2D(128,(3,3),kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2))          
          
model.add(Convolution2D(256,(3,3),kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(Convolution2D(256,(3,3),kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2))

model.add(Convolution2D(512,(3,3),kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(Convolution2D(512,(3,3),kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2),strides=2))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.20))
model.add(Dense(64,kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dropout(0.20))
model.add(Activation('relu'))

model.add(Dense(16,kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Activation('relu'))

model.add(Dense(8,kernel_initializer=my_weight_init,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dense(2,activation='softmax',kernel_initializer=my_weight_init,bias_initializer='zeros'))
          
          
model.summary()      

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 225, 225, 64)      1792      
_________________________________________________________________
batch_normalization_7 (Batch (None, 225, 225, 64)      256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 110, 110, 128)     73856     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 108, 108, 128)     147584    
_________________________________________________________________
batch_normalization_8 (Batch (None, 108, 108, 128)     512       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 54, 54, 128)      

In [44]:
y_train[y_train=="Small"]=0
y_train[y_train=="Large"]=1
print(y_train[:10])

[1 0 0 0 1 1 0 0 0 0]


In [45]:
y_train=to_categorical(y_train)
print(y_train[:5])

[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [46]:
adam=Adam(lr=0.0001)

In [47]:
# for layer in model.layers[:-21]:
#     layer.trainable=False

In [48]:

def generator(batch_size,img_height=227, img_width=227, channels=3, augment=False):

    n_samples = img_train_name.shape[0]
    batch_img = np.zeros((batch_size, img_width, img_height, channels))
    idx = 0
    while True:
        batch_img_path = img_train_name[idx:idx+batch_size]
        for i, p in zip(range(batch_size), batch_img_path):
            img = image.load_img(p, target_size=(img_height, img_width))
            img = image.img_to_array(img)/255.0
            batch_img[i] = img

        target = y_train[idx:idx+batch_size]

        idx += batch_size
        if idx == n_samples:
            idx = 0
        elif idx > n_samples - batch_size:
            idx = n_samples - batch_size


        yield batch_img, target


In [49]:
#def validatn_generator(batch_size,img_height=227, img_width=227, channels=3,):
#     n_samples = 2000
#     batch_img = np.zeros((batch_size, img_width, img_height, channels))
#     idx = 0
#     while True:
#         batch_img_path = img_train_name[val_random[idx+batch_size]]
#         for i, p in zip(range(batch_size), batch_img_path):
#             img = image.load_img(p, target_size=(img_height, img_width))
#             img = image.img_to_array(img)
#             batch_img[i] = img

#         target = y_train[val_random[idx+batch_size]]

#         idx += batch_size
#         if idx == n_samples:
#             idx = 0
#         elif idx > n_samples - batch_size:
#             idx = n_samples - batch_size

#         yield batch_img, target

In [50]:
train_gen = generator(64)
# val_gen=validatn_generator(64)
model.compile(optimizer=adam,loss="binary_crossentropy",metrics=["acc"])

In [51]:

model.fit_generator(train_gen,epochs=8,steps_per_epoch=187)

Epoch 1/8
187/187 [==============================] - 103s 552ms/step - loss: 0.4510 - acc: 0.7886
Epoch 2/8
187/187 [==============================] - 85s 454ms/step - loss: 0.2471 - acc: 0.9150
Epoch 3/8
187/187 [==============================] - 85s 453ms/step - loss: 0.1824 - acc: 0.9448
Epoch 4/8
187/187 [==============================] - 85s 455ms/step - loss: 0.1326 - acc: 0.9667
Epoch 5/8
187/187 [==============================] - 85s 453ms/step - loss: 0.1043 - acc: 0.9742
Epoch 6/8
187/187 [==============================] - 84s 451ms/step - loss: 0.0826 - acc: 0.9809
Epoch 7/8
187/187 [==============================] - 85s 453ms/step - loss: 0.0746 - acc: 0.9814
Epoch 8/8
187/187 [==============================] - 84s 449ms/step - loss: 0.0581 - acc: 0.9872


In [52]:
del y_train,img_train_name

In [53]:
os.listdir("../input/lunar-images/DataSet/Test Images/Test Images")

['lgl60 (343).png',
 'lgl85 (191).png',
 'lg566 (417).png',
 'lg898 (212).png',
 'lg76 (111).png',
 'lg171 (423).png',
 'lgl85 (364).png',
 'lg99 (252).png',
 'lgl125 (548).png',
 'lg170 (185).png',
 'lg171 (106).png',
 'lgl 10 (1).png',
 'lg101 (281).png',
 'lg99 (83).png',
 'lg170 (15).png',
 'lg566 (160).png',
 'lg167 (10).png',
 'lg898 (277).png',
 'lg432 (309).png',
 'lg 988 (96).png',
 'lg171 (251).png',
 'lg 988 (236).png',
 'lg 988 (276).png',
 'lg56 (124).png',
 'lg45 (365).png',
 'lg171 (182).png',
 'lg 988 (334).png',
 'lgl305 (5).png',
 'lg87 (7).png',
 'lg168 (373).png',
 'lg101 (269).png',
 'lg101 (195).png',
 'lg168 (241).png',
 'lg432 (395).png',
 'lg170 (193).png',
 'lg87 (97).png',
 'lgl85 (344).png',
 'lgl60 (106).png',
 'lgl60 (231).png',
 'lg76 (58).png',
 'lg45 (278).png',
 'lg87 (435).png',
 'lg171 (57).png',
 'lg 988 (268).png',
 'lgl125 (30).png',
 'lg45 (486).png',
 'lgl305 (138).png',
 'lg87 (110).png',
 'lg56 (74).png',
 'lg168 (335).png',
 'lg168 (407).png'

In [54]:
img_test=pd.read_csv('../input/lunar-images/DataSet/test.csv')

In [55]:
img_test=img_test['Image_File'].values

In [56]:
img_test.shape

(7534,)

In [57]:
count=0
dir_path="../input/lunar-images/DataSet/Test Images/Test Images/"
for i in img_test:
    img_test[count]=dir_path+i
    count=count+1

In [58]:
img_test[0]

'../input/lunar-images/DataSet/Test Images/Test Images/lg 988 (1).png'

In [81]:
y_final=np.zeros((len(img_test),2))
print(y_final.shape)

(7534, 2)


In [82]:
l=y_final.shape[0]//3
for i in range(3):
    if i==2:
        test_data=np.array([np.array(image.load_img(p,target_size=(227,227,3)))/255.0 for p in img_test[i*l:]])
        y_final[i*l:]=model.predict(test_data)
    else:   
        print(i*l)
        print((i+1)*l)
        test_data=np.array([np.array(image.load_img(p,target_size=(227,227,3)))/255.0 for p in img_test[i*l:(i+1)*l]])
        y_final[i*l:(i+1)*l]=model.predict(test_data)

0
2511
2511
5022


In [83]:
y_final[-1]
y_final=np.argmax(y_final,axis=1)

In [96]:
out=[]
for i in y_final:
    if i==0:
        out.append("Small")
    else:
        out.append("Large")

In [62]:
# out=list(y_final)
# out[out==0]="Small"
# out[out==1]="Large"

In [97]:
out_csv=pd.read_csv("../input/lunar-images/DataSet/test.csv")

In [98]:
out_csv["Class"]=out

In [99]:
out_csv

,Image_File,Class
0,lg 988 (1).png,Small
1,lg 988 (10).png,Small
2,lg 988 (100).png,Large
3,lg 988 (101).png,Small
4,lg 988 (102).png,Small
...,...,...
7529,lg99 (95).png,Large
7530,lg99 (96).png,Large
7531,lg99 (97).png,Large
7532,lg99 (98).png,Large


In [101]:
df=pd.DataFrame(out_csv)

In [111]:
df.to_csv("output.csv",index=False)

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("./output.csv")

In [6]:
df["Class"][:len(df["Class"])//2]=["Small" for i in df['Class'][:len(df['Class'])//2]]

In [12]:
df.to_csv("output2.csv",index=False)

In [13]:
import matplotlib.pyplot as plt